In [55]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

def get_id(path):
    return Path(path).with_suffix('').name.replace('11ph11','/')

def get_dataset(europeana_id):
    return '90402_M_NL_Rijksmuseum' if europeana_id in data1_df['europeana_id'].values else '09102_Ag_EU_MIMO'


In [56]:
data1_path = '/home/jcejudo/projects/duplicate_detection/data/90402_M_NL_Rijksmuseum.csv'
data2_path = '/home/jcejudo/projects/duplicate_detection/data/09102_Ag_EU_MIMO.csv'

path = '/home/jcejudo/projects/duplicate_detection/results/90402_M_NL_Rijksmuseum/similarity.csv'

df = pd.read_csv(path)
df = df.sort_values(by=['distance'],ascending=False)
print(df.shape[0])
df = df.iloc[::2]
print(df.shape[0])

data1_df = pd.read_csv(data1_path)
data1_df = data1_df[['europeana_id','uri','image_url']]

data2_df = pd.read_csv(data2_path)
data2_df = data2_df[['europeana_id','uri','image_url']]


df['from'] = df['from'].apply(get_id)
df['to'] = df['to'].apply(get_id)
df['dataset_image_1'] = df['from'].apply(get_dataset)
df['dataset_image_2'] = df['to'].apply(get_dataset)


1108225
554113


In [ ]:
threshold_list = np.linspace(0.8, 1.0, num=100)
n_pairs_list = []
for threshold in threshold_list:
    _df = df.loc[df['distance'] >= threshold]
    n_pairs = _df.shape[0]
    n_pairs_list.append(n_pairs)
        
fix,ax = plt.subplots(figsize = (8,6))
ax.plot(threshold_list,n_pairs_list)
ax.set_ylabel('# of duplicate pairs')
ax.set_xlabel('Distance threshold')
plt.tight_layout()
plt.show()

In [ ]:
threshold = 0.9

_df = df.copy()
similarity_df = _df.loc[_df['distance'] >= threshold]

print(similarity_df.shape[0])
print(similarity_df.head(5))

In [ ]:
data_df = pd.concat([data1_df,data2_df])

similarity_df = similarity_df.merge(data_df,left_on='from',right_on='europeana_id')
similarity_df = similarity_df.rename(columns={'from':'id1','image_url':'url1','uri':'uri1'})
similarity_df.pop('europeana_id')

similarity_df = similarity_df.merge(data_df,left_on='to',right_on='europeana_id')
similarity_df = similarity_df.rename(columns={'to':'id2','image_url':'url2','uri':'uri2'})
similarity_df.pop('europeana_id')



# _df2 = _df.merge(data2_df,left_on='from',right_on='europeana_id')
# _df2 = _df2.rename(columns={'from':'id1','image_url':'url1','uri':'uri1'})
# _df2.pop('europeana_id')
# _df2 = _df2.merge(data2_df,left_on='to',right_on='europeana_id')
# _df2 = _df2.rename(columns={'to':'id2','image_url':'url2','uri':'uri2'})
# _df2.pop('europeana_id')


print(similarity_df.shape[0])

#_df.pop('distance')


#_df.head(5)

In [ ]:
saving_path = '/home/jcejudo/projects/duplicate_detection/results/90402_M_NL_Rijksmuseum/duplicates.csv'
similarity_df.to_csv(saving_path,index = False)